In [ ]:
from collections import namedtuple
import dataclasses as dc
import pathlib
import random
import uuid

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pendulum
import seaborn as sns

In [ ]:
from orchid import (project_store as ops)

In [ ]:
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
import UnitsNet
# noinspection PyUnresolvedReferences
from System import DateTime

In [ ]:
project_filenames = {
    'bakken': 'frankNstein_Bakken_UTM13_FEET.ifrac',
    'montney': 'Project-frankNstein_Montney_UTM13_METERS.ifrac',
}

In [ ]:
test_data_path = pathlib.Path('c:/src/Orchid.IntegrationTestData/')
project_path_names = toolz.valmap(lambda fn: test_data_path.joinpath(fn), 
                                  project_filenames)
project_path_names

In [ ]:
projects = toolz.valmap(
    lambda pn: ops.ProjectStore(str(pn)).native_project(),
    project_path_names)
projects

In [ ]:
def get_project_monitors(proj):
    return proj.Monitors.Items

project_monitors = toolz.pipe(
    projects,
    toolz.valmap(get_project_monitors),
)
project_monitors

In [ ]:
toolz.pipe(project_monitors.values(),
           toolz.map(len),
           list,)

In [ ]:
@dc.dataclass
class MonitorBasics:
    object_id: uuid.UUID
    name: str
    display_name: str
    start_time: pendulum.DateTime
    stop_time: pendulum.DateTime

@toolz.curry
def summarize_monitor_basics(m):
    result = MonitorBasics(m.ObjectId, m.DisplayName, m.Name, m.StartTime.ToString('o'), m.StopTime.ToString('o'))
    return result

project_monitor_summaries = toolz.pipe(
    project_monitors,
    toolz.valmap(toolz.map(summarize_monitor_basics)),
    toolz.valmap(list),
)
project_monitor_summaries

In [ ]:
selected_field_name = 'bakken'

In [ ]:
def make_id_summary(field_name, summaries):
    field_name_summaries = summaries[field_name]
    return {
        'field': field_name,
        'Object Id': [s.object_id for s in field_name_summaries],
        # 'Display Name': [s.display_name for s in field_name_summaries],
        # 'Name': [s.name for s in field_name_summaries],
        'Start Time': [s.start_time for s in field_name_summaries],
        'Stop Time': [s.stop_time for s in field_name_summaries],
    }

In [ ]:
pd.DataFrame(data=make_id_summary('bakken', project_monitor_summaries))

In [ ]:
pd.DataFrame(data=make_id_summary('montney', project_monitor_summaries))

In [ ]:
@dc.dataclass
class PyTick:
    sample_at: DateTime
    sample_value: float
        
MonitorSummary = namedtuple('MonitorSummary', ['object_id',
                                               'name', 
                                               'display_name',
                                               'time_series_id',
                                               'sample_count',
                                               'samples'])
        
def monitors_time_series(ms):
    result = toolz.pipe(
        ms,
        toolz.map(lambda m:  MonitorSummary(object_id=m.ObjectId.ToString(), 
                                            name=m.Name, 
                                            display_name=m.DisplayName,
                                            time_series_id=str(m.TimeSeries.ObjectId), 
                                            sample_count=len(m.TimeSeries.GetOrderedTimeSeriesHistory()), 
                                            samples=m.TimeSeries.GetOrderedTimeSeriesHistory())),
        lambda summary: sorted(summary, key=lambda ms: ms.display_name),
        list,
    )
    return result

project_monitor_time_series = toolz.pipe(
    project_monitors,
    toolz.valmap(monitors_time_series),
    toolz.valmap(toolz.map(lambda ms: {ms.display_name: ms})),
    toolz.valmap(toolz.merge),
#     toolz.valmap(toolz.map(toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('s'),
#                                                          sample_value=tick.Value)))),
#     toolz.valmap(toolz.map(list)),
#     toolz.valmap(list),
)
project_monitor_time_series

In [ ]:
random.choice([22, 36, 50])

In [ ]:
toolz.pipe(
    project_monitor_time_series,
    toolz.get_in(['bakken', 'Demo_1H - stage 50']),
    lambda ms: toolz.get([0, 6078, 24489, 35902, 47331, 56567, 94551, 114594], ms.samples),
#     lambda ms: toolz.get([0, 1105, 14475, 18418, 33263, 83966, 87110, 114594], ms.samples),
    toolz.map(lambda tick: PyTick(sample_at=tick.Timestamp.ToString('s'), 
                                  sample_value=tick.Value)),
    list,
)